In [1]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler 
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import pickle

##

C:\Users\Chuck B\Anaconda3\envs\py27\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Load and Divide Data into Training and Testing  
  
In addition, the data is scaled. 


In [41]:
# Load comma delimited sensor data into Pandas data frame
sensor_file = "../data/sensor_11_12_2017_10_52_24.csv"
df = pd.read_csv(sensor_file)
print "Dataframe df shape", df.shape
# df.iloc[:,1:16].head()
# df.iloc[:,16].head()

# Extract features and target (y)
X = df.iloc[:,1:16] # exclude date
y = df.iloc[:,16] # state
print "X y shape", X.shape, y.shape

# Use 80% of data for training, 20% for testing
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.20, random_state = 1)
print "X train shape", X_train.shape
print "X test shape", X_test.shape

# Scaling features, mean = 0, stddev = 1
scaler = StandardScaler()
scaler.fit(X_train)  
X_train = scaler.transform(X_train)

# Head
df.head()

##

Dataframe df shape (5457, 17)
X y shape (5457, 15) (5457L,)
X train shape (4365, 15)
X test shape (1092, 15)


,DateTime,Temp,Humidity,Pressure,Yaw,Pitch,Roll,MagX,MagY,MagZ,AccX,AccY,AccZ,GyroX,GyroY,GyroZ,State
0,11/12/2017 10:52:35.251176,29.361,40.702,1031.956,215.801975,356.730118,2.171165,-5.351357,5.945352,26.308798,0.056247,0.034719,0.960166,-0.060679,0.072064,0.049957,standing
1,11/12/2017 10:52:35.523388,29.308,39.749,1031.952,216.099403,356.626506,2.103062,-10.720130,12.090914,53.852757,0.055035,0.016267,0.974544,0.011399,0.019521,-0.016927,standing
2,11/12/2017 10:52:35.688238,29.468,40.231,1031.941,216.340904,356.301112,2.310486,-13.004756,14.781075,65.216164,0.046307,0.014810,0.939451,0.040319,0.005237,-0.007642,standing
3,11/12/2017 10:52:35.849093,29.397,40.121,1031.950,216.613607,356.519507,2.298305,-13.880388,16.216396,69.694740,0.060854,0.016267,0.945544,-0.005668,0.020673,0.007911,standing
4,11/12/2017 10:52:36.010403,29.343,40.088,1031.943,216.940253,356.745464,2.196436,-13.895806,16.390114,71.753525,0.055763,0.023308,0.937014,-0.008612,0.015847,0.017006,standing


# Construct Neural Network Model and Tweak

In [42]:
# Supervised neural network model
# http://scikit-learn.org/stable/modules/neural_networks_supervised.html

# NN has 3 hidden layers. 1st layer has 15 neurons which equals 15 features
#     2nd layer has 100 (just because), and 3rd layer has 8 which equals 8 classes

# Pipeline, push scaler and classifer
pipe = Pipeline([('scl', StandardScaler()),
                 ('clf', MLPClassifier(solver='lbfgs', alpha=0.00001,
                                           hidden_layer_sizes=(15,100, 8), random_state=1))])

# How well does the model converge? Overfitting? Underfitting?
scores = cross_val_score(estimator=pipe, X=X_train, y=y_train,cv=10,n_jobs=1)
for s in scores:
    print (s)
    
# The final CV acc for training data was 0.64 using Logistic Regression
print ('CV Acc: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

##

0.904545454545
0.897494305239
0.929223744292
0.908675799087
0.91095890411
0.924137931034
0.905747126437
0.926267281106
0.910138248848
0.923963133641
CV Acc: 0.914 +/- 0.010


In [14]:
## Show model parameters
pipe.get_params

<bound method Pipeline.get_params of Pipeline(steps=[('scl', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(15, 100, 8), learning_rate='constant',
       learning_r...      solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False))])>

# Use Models to Make Predictions and Evaluate

In [50]:
# Fit training data
pipe.fit(X_train, y_train)
predictions = pipe.predict(scaler.transform(X_test)) # Get predictions

# Better table
pd.crosstab(y_test, predictions)

# Ideally, there would zeros everywhere except for the main diagonal
# Last row, for example, the actual state is 'walking', but the model
# is predicting 'turn-right'


col_0,go_down,go_up,spin_left,spin_right,standing,turn_left,turn_right,walking
State,,,,,,,,
go_down,35,7,0,0,0,3,1,3
go_up,5,45,1,0,1,1,0,0
spin_left,0,0,100,0,0,3,1,0
spin_right,0,0,0,66,1,0,0,0
standing,0,0,1,0,128,0,0,1
turn_left,1,1,1,0,0,157,2,15
turn_right,0,0,0,0,0,2,143,12
walking,6,0,1,0,2,9,9,328


In [51]:
# Metrics
precision, recall, fscore, support = score(y_test, predictions)

print ('Prec ','Rec','  Fscore')
for i in range(len(precision)):
    print (('%.3f' % precision[i]), ('%.3f' % recall[i]), ('%.3f' % fscore[i]))
    
## Note that accuracy is not appropriate for unbalanced classes.
## F-score is the superior metric
## The lowest Fscores are associated with 'go_down' and 'go_up'

##

('Prec ', 'Rec', '  Fscore')
('0.745', '0.714', '0.729')
('0.849', '0.849', '0.849')
('0.962', '0.962', '0.962')
('1.000', '0.985', '0.992')
('0.970', '0.985', '0.977')
('0.897', '0.887', '0.892')
('0.917', '0.911', '0.914')
('0.914', '0.924', '0.919')


# Deploying Demonstration  
  
The following code demonstrate how the Neural Network model is saved and deployed to automatically classify sensor readings into one of eight classes.  

I intend on loading this model to the Raspberry Pi and feeding it with the sensor data to indicate automatically what motion the user is making.


In [52]:
# Use pickle library to save model
filename = '../model/sensor_model_2017.11.20c.sav'
pickle.dump(pipe, open(filename, 'wb'))


In [55]:
# NOTE: For reference only
#Add this to data logger for automatically describing the state of motion
filename = '../model/sensor_model_2017.11.20c.sav'

# Load our saved model
loaded_model = pickle.load(open(filename, 'rb'))
# print loaded_model.get_params

# Scaling features, mean = 0, stddev = 1
scaler = StandardScaler()
scaler.fit(X_test)

# Simulated received data
dat = ['33.302', '34.442', '1018.687', '166.270478', '359.305019', '0.341884', '-19.774937', '-5.494294', '60.614586', '0.020365', '0.000971', '0.947250', '0.001709', '0.002304', '0.002999']

# Make prediction on simulated data
prediction = loaded_model.predict(pd.Series(dat).reshape(1,-1))
print (prediction)

## It may be necessary to calculate a running average on sensors to minimize
## noise.

##

<class 'pandas.core.frame.DataFrame'>
['turn_left']


C:\Users\Chuck B\Anaconda3\envs\py27\lib\site-packages\ipykernel\__main__.py:17: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


# Code

In [70]:
mydata = [['11.406', '22.902', '514.395', '81.636', '179.822', '179.898', '-14.837', '-4.425', '34.947', '0.004', '-0.003', '0.471', '-0.000', '-0.001', '-0.001'],
['11.409', '22.982', '514.394', '81.668', '179.818', '179.895', '-14.865', '-4.491', '34.961', '0.003', '-0.003', '0.472', '-0.000', '-0.000', '-0.001'],
['11.418', '22.867', '514.394', '81.683', '179.817', '179.892', '-14.835', '-4.611', '34.928', '0.003', '-0.003', '0.472', '-0.000', '-0.001', '-0.001']]

# mydata = []
for i in range(0, len(mydata)):
    mydata[i] = map(float, mydata[i])

# print mydata

# # header = ['Temp','Humidity','Pressure','Yaw','Pitch','Roll','MagX','MagY','MagZ','AccX','AccY','AccZ','GyroX',' GyroY','GyroZ']
my_df = pd.DataFrame(mydata)
print type(my_df.mean(axis=0))

# print my_df.mean(axis=0)


<class 'pandas.core.series.Series'>


In [64]:
mylist =['11.406', '22.902', '514.395', '81.636', '179.822', '179.898', '-14.837', '-4.425', '34.947', '0.004', '-0.003', '0.471', '-0.000', '-0.001', '-0.001']
map(float, mylist)

[11.406,
 22.902,
 514.395,
 81.636,
 179.822,
 179.898,
 -14.837,
 -4.425,
 34.947,
 0.004,
 -0.003,
 0.471,
 -0.0,
 -0.001,
 -0.001]